Load all the modules and data.

In [1]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import glob
import os
import sandwell
import emapex
from my_savefig import my_savefig
import gravity_waves as gw

Float = emapex.load(4977, apply_w=False,
    apply_strain=False, apply_iso=False, post_process=True,
    neutral_density=False, regrid=False, verbose=False)

# Bathymetry file path.
bf = os.path.abspath(glob.glob('/noc/users/jc3e13/storage/smith_sandwell/topo_*.img')[0])
# Figure save path.
sdir = '../figures/wave_generation_analysis'
if not os.path.exists(sdir):
    os.makedirs(sdir)
# Universal figure font size.
matplotlib.rc('font', **{'size': 8})


Initialising
------------

EM-APEX float: 4977
Loading data...
All numerical data appears to have been loaded successfully.

Creating array of half profiles.


Post processing
---------------



/nerc/packages/anaconda/4.1.0/lib/python2.7/site-packages/numpy/ma/core.py:790: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)
/nerc/packages/anaconda/4.1.0/lib/python2.7/site-packages/numpy/ma/core.py:802: RuntimeWarning: invalid value encountered in less_equal
  return umath.less_equal(x, self.critical_value)


Estimating absolute velocity and subsurface position.


/nerc/packages/anaconda/4.1.0/lib/python2.7/site-packages/gsw/gibbs/conversions.py:60: RuntimeWarning: invalid value encountered in less
  cond1 = ((p < 100) & (SP > 120))
/nerc/packages/anaconda/4.1.0/lib/python2.7/site-packages/gsw/gibbs/conversions.py:60: RuntimeWarning: invalid value encountered in greater
  cond1 = ((p < 100) & (SP > 120))
/nerc/packages/anaconda/4.1.0/lib/python2.7/site-packages/gsw/gibbs/conversions.py:61: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = ((p >= 100) & (SP > 42))
/nerc/packages/anaconda/4.1.0/lib/python2.7/site-packages/gsw/gibbs/conversions.py:61: RuntimeWarning: invalid value encountered in greater
  cond2 = ((p >= 100) & (SP > 42))
/nerc/packages/anaconda/4.1.0/lib/python2.7/site-packages/gsw/gibbs/conversions.py:428: RuntimeWarning: invalid value encountered in less
  invalid = np.logical_and(p < 100, np.logical_or(t > 80, t < -12))
/nerc/packages/anaconda/4.1.0/lib/python2.7/site-packages/gsw/gibbs/conversions.py:428: Run


Updating half profiles.

Could not calculate neutral density because pygamman 
package could not be imported.


In [15]:
def w_model(params, data):

    phi_0, X, Y, Z, phase_0 = params

    time, x, y, z, U, V, N, f = data

    k = 2*np.pi/X
    l = 2*np.pi/Y
    m = 2*np.pi/Z

    om = gw.omega(N, k, m, l, f)

    w = gw.w(x, y, z, time, phi_0, k, l, m, om, N, U=U, V=V, phase_0=phase_0)

    return w

x = np.linspace(5000., 15000., 50)
z = np.linspace(-1500, -500, 50)
xg, zg = np.meshgrid(x, z)
xg_km = xg/1000.
y = 0.
t = 0.
U = 0.
V = 0.
N = 2.2e-3
f = 1e-4

phi_0 = 0.03
X = 4000.
Y = 1000.
Z = 1000.
phase_0 = 0.

wg = w_model((phi_0, X, Y, Z, phase_0), (t, xg, y, zg, U, V, N, f))

In [16]:
hpids = np.arange(19, 35) 

__, idxs = Float.get_profiles(hpids, ret_idxs=True)

z = Float.z[:, idxs].flatten(order='F')
d = Float.dist_ctd[:, idxs].flatten(order='F')
tgps = Float.UTC_start[idxs]
lon = Float.lon_start[idxs]
lat = Float.lat_start[idxs]
tctd = Float.UTC[:, idxs].flatten(order='F')
nans = np.isnan(d) | np.isnan(tctd)
tctd = tctd[~nans]
dctd = d[~nans]
lonctd = np.interp(tctd, tgps, lon)
latctd = np.interp(tctd, tgps, lat)
bathy = sandwell.interp_track(lonctd, latctd, bf)

d -= dctd[bathy.argmax()]

nans = np.isnan(d)

dctd -= dctd[bathy.argmax()]

fig = plt.figure(figsize=(3.125, 3))
plt.fill_between(dctd[::50], bathy[::50], np.nanmin(bathy), color='black',
                 linewidth=2)
plt.pcolormesh(xg_km, zg, wg, cmap=plt.get_cmap('bwr'))
plt.plot(d, z, color='grey', linewidth=3)

plt.xlabel('Distance from ridge top (km)', labelpad=0.06)
plt.ylabel('$z$ (m)')
plt.ylim(-3500., 0.)
plt.xlim(-5., 25.)

my_savefig(fig, 'summary_w_section', sdir, ftype=('png'), fsize='single_col')